# IEEE39 - EMT/DP Simulation

In [ ]:
import subprocess, sys, os
import urllib.request

dpsim_root_dir = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')
sys.path.insert(0, os.path.join(dpsim_root_dir, 'python/src/dpsim/'))
sys.path.insert(0, os.path.join(dpsim_root_dir, 'build'))

import matpower
import dpsimpy
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import urllib.request
import matplotlib.pyplot as plt

%matplotlib widget

In [ ]:
if not os.path.exists('ieee39-data'):
    os.mkdir('ieee39-data')

file_names = ["20140528T0800Z_XX_YYY_DY_.xml", "20140528T0800Z_XX_YYY_SSH_.xml", "20140528T0800Z_XX_YYY_SV_.xml", "20140528T0800Z_XX_YYY_TP_.xml", "20140528T0800Z_YYY_EQ_.xml"]
for file_name in file_names:
    url_dynamic = 'https://raw.githubusercontent.com/martinmoraga/dpsim_data/main/IEEE39/CIM/' + file_name
    local_file = './ieee39-data/' + file_name
    urllib.request.urlretrieve(url_dynamic, local_file)
    
path_to_files = []
for item in file_names: path_to_files.append('./ieee39-data/' + item)

### PF Simulation for initialization

In [ ]:
sim_name_pf = "IEEE39"

dpsimpy.Logger.set_log_dir('logs/' + sim_name_pf)
reader = dpsimpy.CIMReader(sim_name_pf, dpsimpy.LogLevel.debug, dpsimpy.LogLevel.debug)
system_pf = reader.loadCIM(60, path_to_files, dpsimpy.Domain.SP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.PVNode)
gen_ref = system_pf.component("G 02")
gen_ref.modify_power_flow_bus_type(dpsimpy.PowerflowBusType.VD)

logger = dpsimpy.Logger(sim_name_pf)
for node in system_pf.nodes:
    logger.log_attribute(node.name() + '.V', 'v', node)
    logger.log_attribute(node.name() + '.S', 's', node)
    
sim = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.debug)
sim.set_system(system_pf)
sim.set_time_step(0.1)
sim.set_final_time(0.1)
sim.set_domain(dpsimpy.Domain.SP)
sim.set_solver(dpsimpy.Solver.NRP)
sim.do_init_from_nodes_and_terminals(True)

sim.add_logger(logger)
sim.run()

### Dynamic Simulation with fault at node 7

#### Declare function for dynamic simulation in DPsim

In [ ]:
def IEEE39_Fault(domain="SP", sg_model=dpsimpy.GeneratorType.SG6bOrderVBR, with_fault=True):

    sim_name_dyn = "IEEE39_" + domain 
    dpsimpy.Logger.set_log_dir('logs/' + sim_name_dyn)
    
    dpsim_domain = None
    if domain=="SP":
        dpsim_domain = dpsimpy.Domain.SP
    elif domain=="DP":
        dpsim_domain = dpsimpy.Domain.DP
    else:
        domain="EMT"
        dpsim_domain = dpsimpy.Domain.EMT
        
    reader = dpsimpy.CIMReader(sim_name_dyn, dpsimpy.LogLevel.debug, dpsimpy.LogLevel.debug)
    if (domain=="SP" or domain=="DP"):
        system_dyn = reader.loadCIM(60, path_to_files, dpsim_domain, dpsimpy.PhaseType.Single, sg_model)
    else:
        system_dyn = reader.loadCIM(60, path_to_files, dpsim_domain, dpsimpy.PhaseType.ABC, sg_model)

    # Extend topology with switch
    if with_fault:
        sw = None
        if domain=="SP":
            sw = dpsimpy.sp.ph1.Switch('Fault', dpsimpy.LogLevel.info)
            sw.set_parameters(1e12, 0.1)
            sw.connect([dpsimpy.sp.SimNode.gnd, system_dyn.node('Bus 03')])
        elif domain=="DP":
            sw = dpsimpy.dp.ph1.Switch('Fault', dpsimpy.LogLevel.info)
            sw.set_parameters(1e12, 0.1)
            sw.connect([dpsimpy.dp.SimNode.gnd, system_dyn.node('Bus 03')])
        else:
            sw = dpsimpy.emt.ph3.SeriesSwitch('Fault', dpsimpy.LogLevel.info)
            sw.set_parameters(1e12, 0.1)
            sw.connect([dpsimpy.emt.SimNode.gnd, system_dyn.node('Bus 03')])
        
        sw.open()
        system_dyn.add(sw)
    
    #initialize node voltages using pf results
    system_dyn.init_with_powerflow(system_pf, dpsim_domain)

    # log results
    logger = dpsimpy.Logger(sim_name_dyn)
    for node in system_dyn.nodes:
        logger.log_attribute(node.name()+'.V', 'v', node)
    for gen_name in ["G 01", "G 02", "G 03", "G 04", "G 05", "G 06", "G 07", "G 08", "G 09", "G 10"]:
        logger.log_attribute(gen_name+".Pe", 'Te', system_dyn.component(gen_name))

    # Parametrize and run simulation
    sim = dpsimpy.Simulation(sim_name_dyn, dpsimpy.LogLevel.info)
    sim.set_system(system_dyn)
    if domain=="SP":
        sim.set_time_step(1e-3)
    elif domain=="DP":
        sim.set_time_step(5e-4)
    elif domain=="EMT":
        sim.set_time_step(1e-4)
    sim.set_final_time(5)
    sim.set_domain(dpsim_domain)
    sim.set_solver(dpsimpy.Solver.MNA)
    sim.set_direct_solver_implementation(dpsimpy.DirectLinearSolverImpl.KLU)
    sim.do_init_from_nodes_and_terminals(True)
    sim.add_logger(logger)
    sim.do_system_matrix_recomputation(True)

    # add events
    if with_fault:
        sw_event_1 = dpsimpy.event.SwitchEvent(1, sw, True)
        sw_event_2 = dpsimpy.event.SwitchEvent(1.05, sw, False)
        sim.add_event(sw_event_1)
        sim.add_event(sw_event_2)

    sim.run()
    
    return sim_name_dyn

In [ ]:
sim_name_fault_sp = IEEE39_Fault(domain="SP", sg_model=dpsimpy.GeneratorType.SG4OrderVBR)

In [ ]:
# unstable case (SG order > 4)
sim_name_fault_dp = IEEE39_Fault(domain="DP", sg_model=dpsimpy.GeneratorType.SG6bOrderVBR, with_fault=False)

# stable case (SG order <= 4)
sim_name_fault_dp = IEEE39_Fault(domain="DP", sg_model=dpsimpy.GeneratorType.SG4OrderVBR)

In [ ]:
#sim_name_fault_emt = IEEE39_Fault(domain="EMT", sg_model=dpsimpy.GeneratorType.SG6bOrderVBR)

#### Read DPsim results

In [ ]:
from villas.dataprocessing.timeseries import TimeSeries as ts
import villas.dataprocessing.plottools as pt

dpsim_result_file = 'logs/' + sim_name_fault_sp + '/' + sim_name_fault_sp + '.csv'
ts_dpsim_sp = read_timeseries_csv(dpsim_result_file)

dpsim_result_file = 'logs/' + sim_name_fault_dp + '/' + sim_name_fault_dp + '.csv'
ts_dpsim_dp = read_timeseries_csv(dpsim_result_file)

#dpsim_result_file = 'logs/' + sim_name_fault_emt + '/' + sim_name_fault_emt + '.csv'
#ts_dpsim_emt = read_timeseries_csv(dpsim_result_file)

#### Load DigSilent Results

In [ ]:
if not os.path.exists('ieee39-data'):
    os.mkdir('ieee39-data')

url_active_power = "https://raw.githubusercontent.com/martinmoraga/dpsim_data/main/IEEE39/CIM/results/active_power_ieee39.csv"
urllib.request.urlretrieve(url_active_power, "./ieee39-data/active_power_ieee39.csv")
    
url_node_voltage = "https://raw.githubusercontent.com/martinmoraga/dpsim_data/main/IEEE39/CIM/results/node_voltages_ieee39.csv"
urllib.request.urlretrieve(url_node_voltage, "./ieee39-data/node_voltages_ieee39.csv")

#urllib.request.urlretrieve(url, local_file_3Order) 
ts_digSilent_power = read_timeseries_dpsim("./ieee39-data/active_power_ieee39.csv")
ts_digSilent_voltages = read_timeseries_dpsim("./ieee39-data/node_voltages_ieee39.csv")

### Define plot functions

In [ ]:
timestep_common = 1e-4
t_begin = 0.0
t_end = 4.99
begin_idx = int(t_begin/timestep_common)
end_idx= int(t_end/timestep_common)
time = np.linspace(t_begin, t_end, num=end_idx-begin_idx)

#plot parameters
width = 12
height = 4

def plot_node_volt_abs(varname_dpsim, varname_digSilent, 
                    ts_dpsim_sp, ts_dpsim_dp=None, ts_digSilent=None, nominal_voltage=1, timestep_common=1e-4, y_lim=False):
    
    #convert dpsim voltage to magnitude value and per-unit for comparison with psat
    dpsim_sp_values_abs_pu = ts_dpsim_sp[varname_dpsim].interpolate(timestep_common).abs().values[begin_idx:end_idx] / nominal_voltage
    if ts_dpsim_dp is not None:
        dpsim_dp_values_abs_pu = ts_dpsim_dp[varname_dpsim].interpolate(timestep_common).abs().values[begin_idx:end_idx] / nominal_voltage
    if ts_digSilent is not None:
        digSilent_values = ts_digSilent[varname_digSilent].interpolate(timestep_common).values[begin_idx:end_idx]
    
    plt.figure(figsize=(width, height))
    plt.plot(time, dpsim_sp_values_abs_pu, label='SP - DPsim')
    if ts_dpsim_dp is not None:
        plt.plot(time, dpsim_dp_values_abs_pu, '--', label='DP - DPsim')
    if ts_digSilent is not None:
        plt.plot(time, digSilent_values, '--', label='RMS - DigSilent')
    plt.legend(loc='lower right')
    plt.xlabel('time (s)')
    plt.grid()
    plt.xlim([t_begin, t_end])
    plt.xlabel("time (s)")
    plt.ylabel(varname_digSilent + " (pu)")
    if (y_lim):
        plt.ylim(y_lim)
    plt.show()
    
    #calculate RMSE
    #rmse = np.sqrt(((dpsim_sp_values_abs_pu - psat_values) ** 2).mean())
    #print('RMSE {:s}  = {:.6f} (pu), which is {:.3f}% of the nominal value = {:.3f} (pu) '.format(varname_dpsim, rmse, rmse/1.0*100, 1.0))
    
def plot_elec_power(varname_dpsim, varname_digSilent, 
                    ts_dpsim_sp, ts_dpsim_dp=None, ts_digSilent=None, nominal_power=1, timestep_common=1e-4, y_lim=False):
    
    #convert dpsim voltage to magnitude value and per-unit for comparison with psat
    dpsim_sp_values = ts_dpsim_sp[varname_dpsim].interpolate(timestep_common).values[begin_idx:end_idx]
    if ts_dpsim_dp is not None:
        dpsim_dp_values = ts_dpsim_dp[varname_dpsim].interpolate(timestep_common).values[begin_idx:end_idx]
    if ts_digSilent is not None:
        digSilent_values = ts_digSilent[varname_digSilent].interpolate(timestep_common).values[begin_idx:end_idx] / nominal_power
    
    plt.figure(figsize=(width, height))
    if ts_dpsim_dp is not None:
        plt.plot(time, dpsim_dp_values, label='DP - DPsim')
    plt.plot(time, dpsim_sp_values, label='RMS - DPsim')
    if ts_digSilent is not None:
        plt.plot(time, digSilent_values, '--', label='RMS - DigSilent')
    plt.legend(loc='lower right')
    plt.xlabel('time (s)')
    plt.grid()
    plt.xlim([t_begin, t_end])
    plt.xlabel("time (s)")
    plt.ylabel(varname_dpsim + " (pu)")
    if (y_lim):
        plt.ylim(y_lim)
    plt.show()
    
    #calculate RMSE
    #rmse = np.linalg.norm(dpsim_sp_values - psat_values) / np.sqrt(len(psat_values))
    #print('RMSE {:s}  = {:.6f} (pu), which is {:.3f}% of the nominal value = ~{:.3f} (pu) '.format(varname_dpsim, rmse, rmse/7.0*100, 7.0))

    #return rmse

#### Plot Electrical Power of SG's

In [ ]:
varname_dpsim = 'G 01.Pe'
varname_digSilent = 'G_01'
rmse_g1 = plot_elec_power(varname_dpsim, varname_digSilent, ts_dpsim_sp, ts_dpsim_dp, ts_digSilent_power, nominal_power=10000, timestep_common=1e-4)

In [ ]:
varname_dpsim = 'G 02.Pe'
varname_digSilent = 'G_02'
rmse_g1 = plot_elec_power(varname_dpsim, varname_digSilent, ts_dpsim_sp, ts_dpsim_dp, ts_digSilent_power, nominal_power=700, timestep_common=1e-4)

In [ ]:
varname_dpsim = 'G 03.Pe'
varname_digSilent = 'G_03'
rmse_g1 = plot_elec_power(varname_dpsim, varname_digSilent, ts_dpsim_sp, ts_dpsim_dp, ts_digSilent_power, nominal_power=800, timestep_common=1e-4)

In [ ]:
varname_dpsim = 'G 04.Pe'
varname_digSilent = 'G_04'
rmse_g1 = plot_elec_power(varname_dpsim, varname_digSilent, ts_dpsim_sp, ts_dpsim_dp, ts_digSilent_power, nominal_power=800, timestep_common=1e-4)

In [ ]:
varname_dpsim = 'G 05.Pe'
varname_digSilent = 'G_05'
rmse_g1 = plot_elec_power(varname_dpsim, varname_digSilent, ts_dpsim_sp, ts_dpsim_dp, ts_digSilent_power, nominal_power=600, timestep_common=1e-4)

In [ ]:
varname_dpsim = 'G 06.Pe'
varname_digSilent = 'G_06'
rmse_g1 = plot_elec_power(varname_dpsim, varname_digSilent, ts_dpsim_sp, ts_dpsim_dp, ts_digSilent_power, nominal_power=800, timestep_common=1e-4)

In [ ]:
varname_dpsim = 'G 07.Pe'
varname_digSilent = 'G_07'
rmse_g1 = plot_elec_power(varname_dpsim, varname_digSilent, ts_dpsim_sp, ts_dpsim_dp, ts_digSilent_power, nominal_power=700, timestep_common=1e-4)

In [ ]:
varname_dpsim = 'G 08.Pe'
varname_digSilent = 'G_08'
rmse_g1 = plot_elec_power(varname_dpsim, varname_digSilent, ts_dpsim_sp, ts_dpsim_dp, ts_digSilent_power, nominal_power=700, timestep_common=1e-4)

In [ ]:
varname_dpsim = 'G 09.Pe'
varname_digSilent = 'G_09'
rmse_g1 = plot_elec_power(varname_dpsim, varname_digSilent, ts_dpsim_sp, ts_dpsim_dp, ts_digSilent_power, nominal_power=1000, timestep_common=1e-4)

In [ ]:
varname_dpsim = 'G 10.Pe'
varname_digSilent = 'G_10'
rmse_g1 = plot_elec_power(varname_dpsim, varname_digSilent, ts_dpsim_sp, ts_dpsim_dp, ts_digSilent_power, nominal_power=1000, timestep_common=1e-4)

#### Plot Node Voltages

In [ ]:
varname_dpsim = "Bus 01.V"
varname_digSilent = 'v1'
nominal_voltage = 345000
plot_node_volt_abs(varname_dpsim, varname_digSilent, ts_dpsim_sp, ts_dpsim_dp, ts_digSilent_voltages, nominal_voltage, timestep_common=1e-4)

In [ ]:
varname_dpsim = "Bus 02.V"
varname_digSilent = 'v2'
nominal_voltage = 345000
#plot_node_volt_abs(varname_dpsim, varname_psat, varname_digSilent, ts_dpsim_sp, ts_dpsim_dp, ts_psat, ts_digSilent, nominal_voltage, timestep_common=1e-3, y_lim=[0.6, 1.1])
plot_node_volt_abs(varname_dpsim, varname_digSilent, ts_dpsim_sp, ts_dpsim_dp, ts_digSilent_voltages, nominal_voltage, timestep_common=1e-4)

In [ ]:
varname_dpsim = "Bus 03.V"
varname_digSilent = 'v3'
nominal_voltage = 345000
#plot_node_volt_abs(varname_dpsim, varname_psat, varname_digSilent, ts_dpsim_sp, ts_dpsim_dp, ts_psat, ts_digSilent, nominal_voltage, timestep_common=1e-3, y_lim=[0.6, 1.1])
plot_node_volt_abs(varname_dpsim, varname_digSilent, ts_dpsim_sp, ts_dpsim_dp, ts_digSilent_voltages, nominal_voltage, timestep_common=1e-4)

In [ ]:
varname_dpsim = "Bus 04.V"
varname_digSilent = 'v4'
nominal_voltage = 345000
#plot_node_volt_abs(varname_dpsim, varname_psat, varname_digSilent, ts_dpsim_sp, ts_dpsim_dp, ts_psat, ts_digSilent, nominal_voltage, timestep_common=1e-3, y_lim=[0.6, 1.1])
plot_node_volt_abs(varname_dpsim, varname_digSilent, ts_dpsim_sp, ts_dpsim_dp, ts_digSilent_voltages, nominal_voltage, timestep_common=1e-4)

In [ ]:
varname_dpsim = "Bus 05.V"
varname_digSilent = 'v5'
nominal_voltage = 345000
#plot_node_volt_abs(varname_dpsim, varname_psat, varname_digSilent, ts_dpsim_sp, ts_dpsim_dp, ts_psat, ts_digSilent, nominal_voltage, timestep_common=1e-3, y_lim=[0.6, 1.1])
plot_node_volt_abs(varname_dpsim, varname_digSilent, ts_dpsim_sp, ts_dpsim_dp, ts_digSilent_voltages, nominal_voltage, timestep_common=1e-4)

In [ ]:
varname_dpsim = "Bus 06.V"
varname_digSilent = 'v6'
nominal_voltage = 345000
#plot_node_volt_abs(varname_dpsim, varname_psat, varname_digSilent, ts_dpsim_sp, ts_dpsim_dp, ts_psat, ts_digSilent, nominal_voltage, timestep_common=1e-3, y_lim=[0.6, 1.1])
plot_node_volt_abs(varname_dpsim, varname_digSilent, ts_dpsim_sp, None, ts_digSilent_voltages, nominal_voltage, timestep_common=1e-4)

In [ ]:
varname_dpsim = "Bus 07.V"
varname_digSilent = 'v7'
nominal_voltage = 345000
#plot_node_volt_abs(varname_dpsim, varname_psat, varname_digSilent, ts_dpsim_sp, ts_dpsim_dp, ts_psat, ts_digSilent, nominal_voltage, timestep_common=1e-3, y_lim=[0.6, 1.1])
plot_node_volt_abs(varname_dpsim, varname_digSilent, ts_dpsim_sp, None, ts_digSilent_voltages, nominal_voltage, timestep_common=1e-4)

In [ ]:
varname_dpsim = "Bus 08.V"
varname_digSilent = 'v8'
nominal_voltage = 345000
#plot_node_volt_abs(varname_dpsim, varname_psat, varname_digSilent, ts_dpsim_sp, ts_dpsim_dp, ts_psat, ts_digSilent, nominal_voltage, timestep_common=1e-3, y_lim=[0.6, 1.1])
plot_node_volt_abs(varname_dpsim, varname_digSilent, ts_dpsim_sp, None, ts_digSilent_voltages, nominal_voltage, timestep_common=1e-4)

In [ ]:
varname_dpsim = "Bus 09.V"
varname_digSilent = 'v9'
nominal_voltage = 345000
#plot_node_volt_abs(varname_dpsim, varname_psat, varname_digSilent, ts_dpsim_sp, ts_dpsim_dp, ts_psat, ts_digSilent, nominal_voltage, timestep_common=1e-3, y_lim=[0.6, 1.1])
plot_node_volt_abs(varname_dpsim, varname_digSilent, ts_dpsim_sp, None, ts_digSilent_voltages, nominal_voltage, timestep_common=1e-4)

In [ ]:
varname_dpsim = "Bus 10.V"
varname_digSilent = 'v10'
nominal_voltage = 345000
#plot_node_volt_abs(varname_dpsim, varname_psat, varname_digSilent, ts_dpsim_sp, ts_dpsim_dp, ts_psat, ts_digSilent, nominal_voltage, timestep_common=1e-3, y_lim=[0.6, 1.1])
plot_node_volt_abs(varname_dpsim, varname_digSilent, ts_dpsim_sp, None, ts_digSilent_voltages, nominal_voltage, timestep_common=1e-4)

In [ ]:
varname_dpsim = "Bus 11.V"
varname_digSilent = 'v11'
nominal_voltage = 345000
#plot_node_volt_abs(varname_dpsim, varname_psat, varname_digSilent, ts_dpsim_sp, ts_dpsim_dp, ts_psat, ts_digSilent, nominal_voltage, timestep_common=1e-3, y_lim=[0.6, 1.1])
plot_node_volt_abs(varname_dpsim, varname_digSilent, ts_dpsim_sp, None, ts_digSilent_voltages, nominal_voltage, timestep_common=1e-4)

In [ ]:
varname_dpsim = "Bus 12.V"
varname_digSilent = 'v12'
nominal_voltage = 138000
#plot_node_volt_abs(varname_dpsim, varname_psat, varname_digSilent, ts_dpsim_sp, ts_dpsim_dp, ts_psat, ts_digSilent, nominal_voltage, timestep_common=1e-3, y_lim=[0.6, 1.1])
plot_node_volt_abs(varname_dpsim, varname_digSilent, ts_dpsim_sp, None, ts_digSilent_voltages, nominal_voltage, timestep_common=1e-4)

In [ ]:
#convert dpsim voltage to magnitude value and per-unit for comparison with psat
dpsim_emt_values_abs_pu = ts_dpsim_emt[varname_dpsim+"_0"].interpolate(timestep_common).values[begin_idx:end_idx] / nominal_voltage
digSilent_values = ts_digSilent_voltages[varname_digSilent].interpolate(timestep_common).abs().values[begin_idx:end_idx] * np.sqrt(2/3)

plt.figure(figsize=(width, height))
plt.plot(time, dpsim_emt_values_abs_pu, label='EMT - DPsim')
plt.plot(time, digSilent_values, '--', label='RMS - DigSilent')
plt.legend(loc='lower right')
plt.xlabel('time (s)')
plt.grid()
plt.xlim([t_begin, t_end])
plt.xlabel("time (s)")
plt.ylabel(varname_digSilent + " (pu)")
plt.show()

In [ ]:
#convert dpsim voltage to magnitude value and per-unit for comparison with psat
varname_dpsim = "G 01.Pe"
varname_digSilent = 'G_01'
dpsim_emt_values_abs_pu = ts_dpsim_emt[varname_dpsim].interpolate(timestep_common).values[begin_idx:end_idx]
digSilent_values = ts_digSilent_power[varname_digSilent].interpolate(timestep_common).values[begin_idx:end_idx] / 10000

plt.figure(figsize=(width, height))
plt.plot(time, dpsim_emt_values_abs_pu, label='EMT - DPsim')
plt.plot(time, digSilent_values, '--', label='RMS - DigSilent')
plt.legend(loc='lower right')
plt.xlabel('time (s)')
plt.grid()
plt.xlim([t_begin, t_end])
plt.xlabel("time (s)")
plt.ylabel(varname_digSilent + " (pu)")
plt.show()